In [ ]:
import numpy as np
from scipy.integrate import solve_ivp

def rossler(xyz, a=0.2, b=0.2, c=5.7):
    x, y, z = xyz
    return np.array([-y - z, x + a * y, b + z * (x - c)])

def emb_dm_y(state, a=0.2, b=0.2, c=5.7):
    x, y, z = state
    return np.array([y, x + a * y, -y - z + a * x + a * a * y])

def emb_dm_x(state, a=0.2, b=0.2, c=5.7):
    x, y, z = state
    return np.array([x, -y - z, -x - a * y - b - z * (x - c)])

def emb_dm_z(state, a=0.2, b=0.2, c=5.7):
    x, y, z = state
    return np.array([z, b + z * (x - c), (b + z * (x - c)) * (x - c) + z * (-y - z)])

def emb_dm_yz(state, a=0.2, b=0.2, c=5.7):
    x, y, z = state
    return np.array([
        y + z,
        b + x + a * y - c * z + x * z,
        -b * c + (a + b) * x + (a * a - 1) * y + (c * c - 1) * z - (2 * c + 1) * x * z - z * z + x * x * z
    ])

def emb_dm_zx(state, a=0.2, b=0.2, c=5.7):
    x, y, z = state
    return np.array([
        z + x,
        -y - z + b + z * (x - c),
        -b * (c + 1) + (b - 1) * x - a * y + c * (c + 1) * z + (1 - 2 * c) * x * z - y * z - z * z
    ])

def emb_dm_xy(state, a=0.2, b=0.2, c=5.7):
    x, y, z = state
    return np.array([
        x + y,
        x + (a - 1) * y - z,
        -b + (a - 1) * x + (a * a - a + 1) * y + (c - 1) * z - x * z
    ])

def emb_y_z_y(state, a=0.2, b=0.2, c=5.7):
    x, y, z = state
    return np.array([y, z, x + a * y])

def emb_x_y_x(state, a=0.2, b=0.2, c=5.7):
    x, y, z = state
    return np.array([x, y, -y - z])

def emb_x_y_y(state, a=0.2, b=0.2, c=5.7):
    x, y, z = state
    return np.array([x, y, x + a * y])

def emb_y_z_z(state, a=0.2, b=0.2, c=5.7):
    x, y, z = state
    return np.array([y, z, b + z * (x - c)])

def emb_x_z_z(state, a=0.2, b=0.2, c=5.7):
    x, y, z = state
    return np.array([x, z, b + z * (x - c)])

def emb_x_z_x(state, a=0.2, b=0.2, c=5.7):
    x, y, z = state
    return np.array([x, z, -y - z])

def vizinhos(x, v, epsilon):
    dist = np.sum(np.abs(x - v), axis=1)
    iv = np.where(dist < epsilon)[0]
    return x[iv, :], iv

def svdo(X, p, epsilon):
    S1p_list = []
    N, c = X.shape
    for j in range(0, N, int(p)):
        xv, iv = vizinhos(X, X[j, :], epsilon)
        if len(iv) > c:
            U, S, Vh = np.linalg.svd(xv, full_matrices=False)
            S1p_list.append(S[0] * 100.0 / np.sum(S))
    S1p_arr = np.array(S1p_list)
    return S1p_arr, (np.mean(S1p_arr) / 100.0) / np.std(S1p_arr, ddof=1), len(S1p_list)

def integrate_rossler(t0=0.0, tf=200.0, dt=0.01, x0=None):
    if x0 is None:
        x0 = np.random.randn(3)
    sol = solve_ivp(lambda t, x: rossler(x), [t0, tf], x0, t_eval=np.arange(t0, tf, dt), rtol=1e-7, atol=1e-9)
    return sol.y.T

def run_svdo_for_embedding(emb_func, name, num_iterations=10, N=10000, noise_level=0.0, p=10):
    vals = []
    for k in range(num_iterations):
        X_base = integrate_rossler()[:N, :]
        Y = np.array([emb_func(state) for state in X_base])
        s = Y[:, 0]
        epsilon = 0.1 * (np.max(s) - np.min(s))
        if noise_level > 0:
            Y[:, 0] = s + np.random.randn(*s.shape) * noise_level * np.std(s, ddof=1)
        _, svdo_val, count = svdo(Y, p, epsilon)
        vals.append(svdo_val)
    vals = np.array(vals)
    print(f"{name:12s}: SVDO = {np.mean(vals):.4f} ± {np.std(vals, ddof=1):.4f} (n={len(vals)})")
    return np.mean(vals), np.std(vals, ddof=1)

embeddings = {
    "dm_y":   emb_dm_y,
    "dm_x":   emb_dm_x,
    "dm_z":   emb_dm_z,
    "dm_yz":  emb_dm_yz,
    "dm_zx":  emb_dm_zx,
    "dm_xy":  emb_dm_xy,
    "y_z_y":  emb_y_z_y,
    "x_y_x":  emb_x_y_x,
    "x_y_y":  emb_x_y_y,
    "y_z_z":  emb_y_z_z,
    "x_z_z":  emb_x_z_z,
    "x_z_x":  emb_x_z_x,
}

for name, emb_func in embeddings.items():
    run_svdo_for_embedding(emb_func, name)